# chainer 

## 基本オブジェクト

chainer では順方向に計算を行い、各ノードにその結果などの情報を保持させておき、逆向きにたどることで微分値などを得る。  
そのため、**一度順方向に計算する必要がある** 。

## 必要なモジュールのインポート

In [4]:
import numpy as np
import chainer

from chainer import cuda, Function, gradient_check, \
                                   Variable, optimizers, serializers, utils, \
                                   Link, Chain, ChainList

import chainer.functions as F
import chainer.links as L

##  Variable

### 変数の定義

In [8]:
x1 = Variable(np.array([1]).astype(np.float32))

# Variable クラスから生成

array([ 1.], dtype=float32)

In [9]:
# data 属性で中身見れる
x1.data

array([ 1.], dtype=float32)

### 計算をなぞってみる

まず順方向に計算

In [12]:
z = x1 * 2 + 5
z.data

array([ 7.], dtype=float32)

次に逆方向に計算

In [ ]:
z.backward()

これで微分値が計算されている

In [14]:
x1.grad

array([ 2.], dtype=float32)

## Functions

戻り値も Variable になることに注意

In [16]:
x = Variable(np.array([3.14]).astype(np.float32))

In [17]:
F.cos(x).data

array([-0.99999875], dtype=float32)

In [18]:
F.sigmoid(x).data

array([ 0.9585129], dtype=float32)

微分もできる

In [44]:
x = Variable(np.array([3.14]).astype(np.float32))

In [45]:
z = F.sin(x)
z.data

array([ 0.00159255], dtype=float32)

In [46]:
z.backward()

In [47]:
x.grad

array([-0.99999875], dtype=float32)

In [48]:
# sin'(x) = cos(x) の確認
F.cos(x).data

array([-0.99999875], dtype=float32)

## Links

入力とは別にパラメータをもった関数のこと。  
例えば Linear は下記を表す。
```python
# パラメータは X と b
f(x) = Wx + b
```
下記いいまとめ（実際にコード読むのいいかも）  
http://ailaby.com/chainer_linear/

### Linear 関数

In [49]:
h = L.Linear(2, 3)

最初は W には適当な数、b には 0 が入っている。

In [51]:
h.W.data

array([[ 0.32657489, -1.64277291],
       [ 1.06587768,  0.74474269],
       [-0.21756326, -0.85319376]], dtype=float32)

In [52]:
h.b.data

array([ 0.,  0.,  0.], dtype=float32)

入力はバッチ（データの集合として与える）

In [58]:
x = Variable(np.arange(6).astype(np.float32).reshape(3, 2))
x.data

array([[ 0.,  1.],
       [ 2.,  3.],
       [ 4.,  5.]], dtype=float32)

In [59]:
y = h(x)
y.data

array([[-1.64277291,  0.74474269, -0.85319376],
       [-4.27516937,  4.36598349, -2.99470782],
       [-6.90756464,  7.9872241 , -5.13622189]], dtype=float32)

検証してみる（数式に則って計算してみる）

In [61]:
w = h.W.data
x0 = x.data
x0.dot(w.T) + h.b.data

array([[-1.64277291,  0.74474269, -0.85319376],
       [-4.27516937,  4.36598349, -2.99470782],
       [-6.90756464,  7.9872241 , -5.13622189]], dtype=float32)